# Домашнее задание 2. Основные операторы PostgreSQL

In [1]:
import sys
import os
import psycopg2
import pandas as pd
import warnings

from pathlib import Path
from IPython.display import display
from psycopg2.extras import execute_values, DictCursor
from datetime import datetime

warnings.filterwarnings('ignore', category=UserWarning)

os.environ['PG_CONFIG'] = '/opt/homebrew/bin/pg_config'
!{sys.executable} -m pip install psycopg2-binary

In [2]:
# Connecting to the database
connection = psycopg2.connect(
    host="localhost",
    database="homework_2",
    user="postgres",
    password="iamroot",
    port="5432"
)
cursor = connection.cursor()
print('Получено соединение с БД')

Получено соединение с БД


## Создание таблиц

**Проблема и ее решение.** При первой попытке залить данные в таблицу product, обнаружилась проблема неуникальности продуктов по product_id, соответственно невозможно было создать FK для таблицы order_items.product_id. В итоге возникла необходимость удалить дубликаты из данных по продуктам для сохранения в таблицу product и обеспечения уникальности по product_id. В связи с чем приняла решение создать временную таблицу product_temp для сохранения всех данных из файла product.csv, с последующей выборкой с фильтрацией и пересохранением данных в таблицу product. Затем таблица product_temp была удалена.

**1. customer:**
   
```dbml
CREATE TABLE IF NOT EXISTS customer (
    customer_id BIGINT PRIMARY KEY,
    first_name VARCHAR NOT NULL,
    last_name VARCHAR,
    gender VARCHAR NOT NULL,
    DOB DATE,
    job_title VARCHAR,
    job_industry_category VARCHAR NOT NULL,
    wealth_segment VARCHAR NOT NULL,
    deceased_indicator CHAR(1) NOT NULL,
    owns_car VARCHAR NOT NULL CHECK (owns_car IN ('Yes','No')),
    address TEXT NOT NULL,
    postcode INTEGER NOT NULL,
    state VARCHAR NOT NULL,
    country VARCHAR NOT NULL,
    property_valuation INTEGER NOT NULL
);
```

**2. orders**
   
```dbml
CREATE TABLE IF NOT EXISTS orders (
    order_id BIGINT PRIMARY KEY,
    customer_id BIGINT NOT NULL,
    order_date DATE NOT NULL,
    online_order BOOLEAN,
    order_status VARCHAR NOT NULL CHECK (order_status IN 
    ('Approved', 'Cancelled')
    )
);
```   

**3. product**

```dbml
CREATE TABLE IF NOT EXISTS product (
    product_id BIGINT PRIMARY KEY,
    brand VARCHAR,
    product_line VARCHAR,
    product_class VARCHAR CHECK (product_class IN 
    ('low', 'medium', 'high')
    ),
    product_size VARCHAR CHECK (product_size IN 
    ('small', 'medium', 'large')
    ),
    list_price FLOAT NOT NULL,
    standard_cost FLOAT
);
```

**4. order_items**

```dbml
CREATE TABLE IF NOT EXISTS order_items (
    order_item_id BIGINT NOT NULL,
    order_id BIGINT NOT NULL,
    product_id BIGINT NOT NULL,
    quantity INTEGER NOT NULL,
    item_list_price_at_sale FLOAT NOT NULL,
    item_standard_cost_at_sale FLOAT
);
```

**5. product_temp (временная таблица, после пересохранения данных в product будет удалена)**

```dbml
CREATE TABLE IF NOT EXISTS product_temp (
    product_id BIGINT NOT NULL,
    brand VARCHAR,
    product_line VARCHAR,
    product_class VARCHAR CHECK (product_class IN 
    ('low', 'medium', 'high')
    ),
    product_size VARCHAR CHECK (product_size IN 
    ('small', 'medium', 'large')
    ),
    list_price FLOAT NOT NULL,
    standard_cost FLOAT
);
```

Ниже используется SQL запрос с удалением ограничений на таблицы и удалением таблиц, если они были уже были созданы, перед тем как из создать.

In [3]:
CREATE_TABLE_QUERY = """
ALTER TABLE order_items DROP CONSTRAINT IF EXISTS fk_order_items;
DROP TABLE IF EXISTS order_items;
ALTER TABLE orders DROP CONSTRAINT IF EXISTS fk_order_customer;
DROP TABLE IF EXISTS orders;
DROP TABLE IF EXISTS customer; 
DROP TABLE IF EXISTS product_temp;
DROP TABLE IF EXISTS product;

CREATE TABLE IF NOT EXISTS customer (
    customer_id BIGINT PRIMARY KEY,
    first_name VARCHAR NOT NULL,
    last_name VARCHAR,
    gender VARCHAR NOT NULL,
    DOB DATE,
    job_title VARCHAR,
    job_industry_category VARCHAR NOT NULL,
    wealth_segment VARCHAR NOT NULL,
    deceased_indicator CHAR(1) NOT NULL,
    owns_car VARCHAR NOT NULL CHECK (owns_car IN ('Yes','No')),
    address TEXT NOT NULL,
    postcode INTEGER NOT NULL,
    state VARCHAR NOT NULL,
    country VARCHAR NOT NULL,
    property_valuation INTEGER NOT NULL
);

CREATE TABLE IF NOT EXISTS product_temp (
    product_id BIGINT NOT NULL,
    brand VARCHAR,
    product_line VARCHAR,
    product_class VARCHAR CHECK (product_class IN 
    ('low', 'medium', 'high')
    ),
    product_size VARCHAR CHECK (product_size IN 
    ('small', 'medium', 'large')
    ),
    list_price FLOAT NOT NULL,
    standard_cost FLOAT
);

CREATE TABLE IF NOT EXISTS product (
    product_id BIGINT PRIMARY KEY,
    brand VARCHAR,
    product_line VARCHAR,
    product_class VARCHAR CHECK (product_class IN 
    ('low', 'medium', 'high')
    ),
    product_size VARCHAR CHECK (product_size IN 
    ('small', 'medium', 'large')
    ),
    list_price FLOAT NOT NULL,
    standard_cost FLOAT
);

CREATE TABLE IF NOT EXISTS orders (
    order_id BIGINT PRIMARY KEY,
    customer_id BIGINT NOT NULL,
    order_date DATE NOT NULL,
    online_order BOOLEAN,
    order_status VARCHAR NOT NULL CHECK (order_status IN 
    ('Approved', 'Cancelled')
    )
);

CREATE TABLE IF NOT EXISTS order_items (
    order_item_id BIGINT NOT NULL,
    order_id BIGINT NOT NULL,
    product_id BIGINT NOT NULL,
    quantity INTEGER NOT NULL,
    item_list_price_at_sale FLOAT NOT NULL,
    item_standard_cost_at_sale FLOAT
);
"""

In [4]:
# Creating tables
cursor.execute(CREATE_TABLE_QUERY)
connection.commit()
print('Таблицы созданы')

Таблицы созданы


## Добавление ограничений на внешние ключи

Внешние ключи есть только у таблиц orders (orders.customer_id) и order_items (order_items.order_id, order_items.product_id). Так как сначала данные будут заливаться во все таблицы, кроме product, то и ограничение на внешний ключ order_items.product_id нужно добавить после того, как таблица product будет заполнена. 

**1. orders:**
   
```dbml
ALTER TABLE IF EXISTS orders
ADD CONSTRAINT fk_order_customer
FOREIGN KEY (customer_id) REFERENCES customer(customer_id);
```

**2. order_items:**
   
```dbml
ALTER TABLE IF EXISTS order_items
ADD CONSTRAINT fk_order_items_order
FOREIGN KEY (order_id) REFERENCES orders(order_id);
```

Добавляем ограничения на внешние ключи, кроме order_items.product_id (будет добавлено позже):

In [5]:
ALTER_TABLE_QUERY = """
ALTER TABLE IF EXISTS orders
ADD CONSTRAINT fk_order_customer
FOREIGN KEY (customer_id) REFERENCES customer(customer_id);

ALTER TABLE IF EXISTS order_items
ADD CONSTRAINT fk_order_items_order
FOREIGN KEY (order_id) REFERENCES orders(order_id);
"""

In [6]:
# Adding restrictions on foreign keys
cursor.execute(ALTER_TABLE_QUERY)
connection.commit()
print('Добавлены ограничения на внешние ключи для таблиц orders и order_items')

Добавлены ограничения на внешние ключи для таблиц orders и order_items


## Заполнение данными

Сначала заполним данными все таблицы кроме product.

**Проблема.** При загрузке данных в таблицы orders и order_items возникли проблемы из-за отсутствия внешних ключей некоторых данных в связанных таблицах, поэтому их пришлось удалить из файлов orders.csv и order_items.csv:

```dbml
psycopg2.errors.ForeignKeyViolation: insert or update on table "orders" violates foreign key constraint "fk_order_customer"
DETAIL:  Key (customer_id)=(5034) is not present in table "customer".

psycopg2.errors.ForeignKeyViolation: insert or update on table "order_items" violates foreign key constraint "fk_order_items_order"
DETAIL:  Key (order_id)=(8708) is not present in table "orders".

psycopg2.errors.ForeignKeyViolation: insert or update on table "order_items" violates foreign key constraint "fk_order_items_order"
DETAIL:  Key (order_id)=(16701) is not present in table "orders".

psycopg2.errors.ForeignKeyViolation: insert or update on table "order_items" violates foreign key constraint "fk_order_items_order"
DETAIL:  Key (order_id)=(17469) is not present in table "orders".
```


In [7]:
data_to_insert = {'customer': ';', 'orders': ',', 'product': ',', 'order_items': ','}

for table_name, delimiter_for_table in data_to_insert.items():
    data = pd.read_csv(
        Path(f'data_to_insert/{table_name}.csv'),
        keep_default_na=False,
        delimiter=delimiter_for_table
    )
    if table_name == 'product':
        table_name = 'product_temp'
    insert_query = f'INSERT INTO {table_name} VALUES %s'
    data = data.replace('', None)
    data = data.where(pd.notna(data), None)
    execute_values(cursor, insert_query, [data_list for data_list in data.values.tolist()])

# Fixing changes in the database
connection.commit()

print('Таблицы заполнены данными')

Таблицы заполнены данными


## Наполнение данными с уникальными ключами таблицы product

**1. Фильтрация данных и их сохранение в таблице product:**

```dbml
INSERT INTO product 
SELECT product_id, brand, product_line, product_class, product_size, list_price, standard_cost
FROM (
    SELECT *, row_number() OVER (PARTITION BY product_id ORDER BY list_price DESC) AS rn 
    FROM product_temp 
    ORDER BY product_id, brand, rn
    ) AS temp 
WHERE rn = 1
```

**2. Удаление временной таблицы product_temp и добавление ограничения на внешний ключ order_items.product_id:**

```dbml
DROP TABLE IF EXISTS product_temp;

ALTER TABLE IF EXISTS order_items
ADD CONSTRAINT fk_order_items_product
FOREIGN KEY (product_id) REFERENCES product(product_id);
```

In [8]:
INSERT_TO_PRODUCTION_QUERY = """
INSERT INTO product 
SELECT product_id, brand, product_line, product_class, product_size, list_price, standard_cost
FROM (
    SELECT *, row_number() OVER (PARTITION BY product_id ORDER BY list_price DESC) AS rn 
    FROM product_temp 
    ORDER BY product_id, brand, rn
    ) AS temp 
WHERE rn = 1
"""


DROP_AND_ALTER_TABLE_AFTER_CLEAR_DATA_QUERY = """
DROP TABLE IF EXISTS product_temp;

ALTER TABLE IF EXISTS order_items
ADD CONSTRAINT fk_order_items_product
FOREIGN KEY (product_id) REFERENCES product(product_id);
"""

In [9]:
cursor.execute(INSERT_TO_PRODUCTION_QUERY)
print('Таблица product заполнена данными')

cursor.execute(DROP_AND_ALTER_TABLE_AFTER_CLEAR_DATA_QUERY)
print('Удалена временная таблица product_temp и добавлено ограничение ключа для order_items.product_id')

Таблица product заполнена данными
Удалена временная таблица product_temp и добавлено ограничение ключа для order_items.product_id


## Выполнение запросов

### 1. Вывести все уникальные бренды, у которых есть хотя бы один продукт со стандартной стоимостью выше 1500 долларов, и суммарными продажами не менее 1000 единиц.

Предлагаю вашему вниманию два решения, так как задачу можно понять двояко - условие о суммарных продаж не менее 1000 единиц применимо к самому бренду, либо к продукту со стоимостью свыше 1500 долларов

**Решение 1. суммарными продажами не менее 1000 единиц применительно к brand:**

In [10]:
query_1 = pd.read_sql("""
SELECT brand
FROM order_items oi
INNER JOIN product p ON oi.product_id = p.product_id AND p.standard_cost > 1500
WHERE brand IS NOT NULL
GROUP BY brand
HAVING SUM(quantity) >= 1000 
ORDER BY brand
""", connection)

In [11]:
query_1.head()

,brand
0,Giant Bicycles
1,OHM Cycles
2,Solex
3,Trek Bicycles


**Решение 2. суммарными продажами не менее 1000 единиц применительно к product_id:**

In [12]:
query_1 = pd.read_sql("""
SELECT DISTINCT brand
FROM order_items oi
INNER JOIN product p ON oi.product_id = p.product_id AND p.standard_cost > 1500
WHERE brand IS NOT NULL
GROUP BY brand, oi.product_id
HAVING SUM(quantity) >= 1000
ORDER BY brand
""", connection)
query_1.head()

,brand
0,Giant Bicycles
1,OHM Cycles


### 2. Для каждого дня в диапазоне с 2017-04-01 по 2017-04-09 включительно вывести количество подтвержденных онлайн-заказов и количество уникальных клиентов, совершивших эти заказы.

In [13]:
query_2 = pd.read_sql("""
SELECT order_date, COUNT(DISTINCT customer_id) AS unique_customer, COUNT(oi.order_id) AS order_count
FROM order_items oi
LEFT JOIN orders o ON oi.order_id = o.order_id 
WHERE order_date BETWEEN '2017-04-01' AND '2017-04-09' AND order_status = 'Approved' AND online_order = true
GROUP BY order_date 
ORDER BY order_date 
""", connection)

display(query_2)

,order_date,unique_customer,order_count
0,2017-04-01,37,37
1,2017-04-02,29,29
2,2017-04-03,27,27
3,2017-04-04,32,32
4,2017-04-05,32,33
5,2017-04-06,36,36
6,2017-04-07,24,24
7,2017-04-08,33,33
8,2017-04-09,30,30


### 3. Вывести профессии клиентов:
- из сферы IT, чья профессия начинается с Senior;
- из сферы Financial Services, чья профессия начинается с Lead.
Для обеих групп учитывать только клиентов старше 35 лет. Объединить выборки с помощью UNION ALL.

In [14]:
query_3 = pd.read_sql("""
SELECT DISTINCT job_title 
FROM customer
WHERE job_title LIKE 'Senior%' AND EXTRACT(YEAR FROM AGE(NOW(), '1974-05-12'::DATE)) >= 35
UNION ALL
SELECT DISTINCT job_title
FROM customer
WHERE job_industry_category = 'Financial Services' AND job_title like 'Lead%' 
AND EXTRACT(YEAR FROM AGE(NOW(), '1974-05-12'::DATE)) >= 35
""", connection)

display(query_3)

,job_title
0,Senior Cost Accountant
1,Senior Developer
2,Senior Editor
3,Senior Financial Analyst
4,Senior Quality Engineer
5,Senior Sales Associate


### 4. Вывести бренды, которые были куплены клиентами из сферы Financial Services, но не были куплены клиентами из сферы IT.

Предлагаю два решения, в первом вычесть из множества брендов, продукты которых покупали клиенты из сферы Financial Services множество брендов, продукты которых покупали клиенты из сферы IT, а во втором использовать оконную функцию. В обоих случая списки оказались пустые, так как в имеющихся данных нет таких брендов, продукты которых покупали бы одни, но не покупали другие. 

In [15]:
query_4 = pd.read_sql("""
SELECT brand_for_financial.brand 
FROM (SELECT DISTINCT brand 
FROM order_items oi
INNER JOIN orders o ON oi.order_id = o.order_id AND order_status = 'Approved'
INNER JOIN customer c ON o.customer_id = c.customer_id AND job_industry_category = 'Financial Services'
LEFT JOIN product p ON oi.product_id = p.product_id 
ORDER BY brand) AS brand_for_financial
LEFT JOIN (SELECT DISTINCT brand 
FROM order_items oi
INNER JOIN orders o ON oi.order_id = o.order_id AND order_status = 'Approved'
INNER JOIN customer c ON o.customer_id = c.customer_id AND job_industry_category = 'IT'
LEFT JOIN product p ON oi.product_id = p.product_id 
ORDER BY brand) brand_for_it ON brand_for_financial.brand = brand_for_it.brand 
WHERE brand_for_it.brand IS NULL
""", connection)

display(query_4)

,brand


In [16]:
query_4 = pd.read_sql("""
SELECT brand 
FROM (SELECT brand, job_industry_category
, DENSE_RANK() OVER (PARTITION BY brand ORDER BY job_industry_category DESC) AS dns_rnk
FROM order_items oi 
LEFT JOIN product p ON oi.product_id = p.product_id 
LEFT JOIN orders o ON oi.order_id = o.order_id 
INNER JOIN customer c ON o.customer_id = c.customer_id AND job_industry_category IN ('Financial Services', 'IT')
GROUP BY brand, job_industry_category) AS brand_jobs_industry
WHERE dns_rnk = 1 AND job_industry_category = 'Financial Services'
""", connection)

display(query_4)

,brand


### 5. Вывести 10 клиентов (ID, имя, фамилия), которые совершили наибольшее количество онлайн-заказов (в штуках) брендов Giant Bicycles, Norco Bicycles, Trek Bicycles, при условии, что они активны и имеют оценку имущества (property_valuation) выше среднего среди клиентов из того же штата.

In [17]:
query_5 = pd.read_sql("""
SELECT o.customer_id, first_name, last_name
FROM order_items oi 
LEFT JOIN product p ON oi.product_id = p.product_id 
LEFT JOIN orders o ON oi.order_id = o.order_id 
LEFT JOIN (SELECT customer_id, first_name, last_name, deceased_indicator, property_valuation, state  
,AVG(property_valuation) OVER(PARTITION BY state) AS avg_property_valuation
FROM customer c ) AS c ON o.customer_id  = c.customer_id 
WHERE brand IN ('Giant Bicycles', 'Norco Bicycles', 'Trek Bicycles') AND online_order = true 
AND property_valuation > avg_property_valuation AND deceased_indicator = 'N' 
GROUP BY o.customer_id, first_name, last_name
ORDER BY COUNT(DISTINCT oi.order_id) DESC
LIMIT 10
""", connection)

display(query_5)

,customer_id,first_name,last_name
0,787,Norma,Batrim
1,2595,Land,Bangley
2,1,Laraine,Medendorp
3,1117,Georgena,Guilaem
4,2498,Rosana,Emmatt
5,353,Antonia,Cardis
6,2072,Margie,Tillyer
7,273,Nevile,Abraham
8,1033,Jacob,Claringbold
9,2637,Marcile,Christley


### 6. Вывести всех клиентов (ID, имя, фамилия), у которых нет подтвержденных онлайн-заказов за последний год, но при этом они владеют автомобилем и их сегмент благосостояния не Mass Customer.

Два варианта решения: в одном за последний год принимается текущий последний год, во втором - последний год с даты последнего заказа (так как данные имеются исключительно за 2017 год)

**Решение 1. за последний год принимается последний год с даты последнего заказа по имеющимся данным**

In [18]:
query_6 = pd.read_sql("""
SELECT c.customer_id, first_name, last_name
FROM customer c
LEFT JOIN (
SELECT DISTINCT customer_id
,MAX(order_date) OVER (partition by customer_id) AS customer_max_order_date
,MAX(order_date) OVER () AS total_max_order_date
FROM orders 
WHERE online_order = true AND order_status = 'Approved'
) AS o ON c.customer_id = o.customer_id 
WHERE wealth_segment != 'Mass Customer' AND owns_car = 'Yes' AND (customer_max_order_date IS NULL
OR customer_max_order_date < (total_max_order_date - INTERVAL '1 YEAR'))
""", connection)

print(f'Количество данных в результате: {query_6.shape[0]}')

Количество данных в результате: 173


**Решение 2. за последний год принимается текущий год (соответственно выборка получилась гораздо больше)**

In [19]:
query_6 = pd.read_sql("""
SELECT c.customer_id, first_name, last_name
FROM customer c
LEFT JOIN (SELECT DISTINCT customer_id
,MAX(order_date) OVER (PARTITION BY customer_id) AS customer_max_order_date
FROM orders 
WHERE online_order = true AND order_status = 'Approved'
) AS o ON c.customer_id = o.customer_id 
WHERE wealth_segment != 'Mass Customer' AND owns_car = 'Yes' AND (customer_max_order_date IS NULL
OR customer_max_order_date < (NOW() - INTERVAL '1 YEAR'))
""", connection)

print(f'Количество данных в результате: {query_6.shape[0]}')

Количество данных в результате: 1032


### 7. Вывести всех клиентов из сферы 'IT' (ID, имя, фамилия), которые купили 2 из 5 продуктов с самой высокой list_price в продуктовой линейке Road.

In [20]:
query_7 = pd.read_sql("""
SELECT o.customer_id, first_name, last_name
FROM order_items oi 
INNER JOIN (SELECT * 
FROM product p 
WHERE product_line = 'Road'
ORDER BY list_price DESC 
LIMIT 5) AS p ON oi.product_id = p.product_id
LEFT JOIN orders o ON oi.order_id = o.order_id 
LEFT JOIN customer c ON o.customer_id = c.customer_id 
WHERE job_industry_category = 'IT'
GROUP BY o.customer_id, first_name, last_name
HAVING COUNT(DISTINCT oi.product_id) = 2
ORDER BY customer_id
""", connection)

display(query_7)

,customer_id,first_name,last_name
0,604,Mella,Petrovsky
1,983,Shaylyn,Riggs
2,1683,Brenn,Bacon
3,2469,Kermie,Hedger
4,3406,Lucy,Lackmann


### 8. Вывести клиентов (ID, имя, фамилия, сфера деятельности) из сфер IT или Health, которые совершили не менее 3 подтвержденных заказов в период 2017-01-01 по 2017-03-01, и при этом их общий доход от этих заказов превышает 10 000 долларов.
### Разделить вывод на две группы (IT и Health) с помощью UNION.

In [21]:
query_8 = pd.read_sql("""
(SELECT o.customer_id, first_name, last_name, job_industry_category
FROM order_items oi 
LEFT JOIN orders o ON oi.order_id = o.order_id
LEFT JOIN customer c ON o.customer_id = c.customer_id
WHERE order_status = 'Approved' AND order_date BETWEEN '2017-01-01' AND '2017-03-01' AND job_industry_category = 'IT'
GROUP BY o.customer_id, first_name, last_name, job_industry_category
HAVING COUNT(DISTINCT oi.order_id) >= 3 AND SUM(quantity * item_standard_cost_at_sale) > 10000)
UNION
(SELECT o.customer_id, first_name, last_name, job_industry_category 
FROM order_items oi 
LEFT JOIN orders o ON oi.order_id = o.order_id
LEFT JOIN customer c ON o.customer_id = c.customer_id
WHERE order_status = 'Approved' AND order_date BETWEEN '2017-01-01' AND '2017-03-01' AND job_industry_category = 'Health'
GROUP BY o.customer_id, first_name, last_name, job_industry_category
HAVING COUNT(DISTINCT oi.order_id) >= 3 AND SUM(quantity * item_standard_cost_at_sale) > 10000)
ORDER BY job_industry_category, customer_id
""", connection)

display(query_8)

,customer_id,first_name,last_name,job_industry_category
0,167,Nathalie,Tideswell,Health
1,173,Ebba,Hanselmann,Health
2,250,Kristofer,None,Health
3,424,Dennie,Eunson,Health
4,590,Ddene,Burleton,Health
5,607,Adelaida,Redmond,Health
6,723,Madelina,Marte,Health
7,1154,Chase,Youd,Health
8,1298,Guglielmo,Yele,Health
9,1762,Gaye,Steutly,Health


**И решение без использования UNION:**

In [22]:
query_8 = pd.read_sql("""
SELECT o.customer_id, first_name, last_name, job_industry_category, COUNT(DISTINCT oi.order_id) order_count
FROM order_items oi 
LEFT JOIN orders o ON oi.order_id = o.order_id
LEFT JOIN customer c ON o.customer_id = c.customer_id
WHERE order_status = 'Approved' AND order_date BETWEEN '2017-01-01' AND '2017-03-01' 
AND job_industry_category in ('IT', 'Health')
GROUP BY o.customer_id, first_name, last_name, job_industry_category
HAVING COUNT(DISTINCT oi.order_id) >= 3 AND SUM(quantity * item_standard_cost_at_sale) >= 10000
ORDER BY job_industry_category, o.customer_id
""", connection)

display(query_8)

,customer_id,first_name,last_name,job_industry_category,order_count
0,167,Nathalie,Tideswell,Health,3
1,173,Ebba,Hanselmann,Health,3
2,250,Kristofer,None,Health,3
3,424,Dennie,Eunson,Health,4
4,590,Ddene,Burleton,Health,4
5,607,Adelaida,Redmond,Health,3
6,723,Madelina,Marte,Health,4
7,1154,Chase,Youd,Health,3
8,1298,Guglielmo,Yele,Health,4
9,1762,Gaye,Steutly,Health,3


In [23]:
# Closing the connection
cursor.close()
connection.close()
print('Cоединение с БД закрыто')

Cоединение с БД закрыто
